In [1]:
#For importing data.
import sys
import os
sys.path.append("..")  # Adds higher directory to python modules path.
import pandas as pd
from PIL import Image
import pickle

#For analyzing similarity
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [2]:
#Retrieve Model
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()

#Transforms
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [3]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.squeeze())
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature bvector
    return my_embedding

In [7]:
files = os.listdir('./Nat_Gard_Build/static/pics/')
pic_embs = {}
for plant in files:
    pic_embs[plant] = get_vector('./Nat_Gard_Build/static/pics/'+plant)

In [8]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
pickle.dump(pic_embs, open('./Nat_Gard_Build/models/data.pkl','wb'))